# Logging

To keep track of what is happening in your code, you can use the provided logger instance:

In [1]:
from qsprpred.logs import logger

logger.warning("This is a warning.")

qsprpred - WARNING - This is a warning.


You can see the logger displays its name, the level of the message, the current file with a line number and the message itself. This logger is used throughout the codebase to keep track of what is happening, but feel free to use it for your own code as well just like in the example above. You can set the message level to `debug`, `info`, `warning`, `error` or `critical`. The default level is `warning`, so `debug` and `info` messages will not be displayed. This is useful to know when you want to debug a piece of QSPRpred code:

In [2]:
import os
from qsprpred.data import QSPRDataset

os.makedirs("../../tutorial_output/data", exist_ok=True)
QSPRDataset.fromTableFile(  # no message is displayed
    filename="../../tutorial_data/A2A_LIGANDS.tsv",
    store_dir="../../tutorial_output/data",
    name="LoggingTutorialDataset",
    target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
    random_state=42
)

# change logger for one with a lower level
from qsprpred.logs import setLogger
import logging

logger.setLevel(logging.DEBUG)
setLogger(logger)

QSPRDataset.fromTableFile(  # more information is shown
    filename="../../tutorial_data/A2A_LIGANDS.tsv",
    store_dir="../../tutorial_output/data",
    name="LoggingTutorialDataset",
    target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
    random_state=42
)

qsprpred - WARNING - Existing data set found, but also found a data frame in store. Refusing to overwrite data. If you want to overwrite data in store, set overwrite=True.
qsprpred - INFO - Loading data from ../../tutorial_data/A2A_LIGANDS.tsv...
qsprpred - WARNING - Existing data set found, but also found a data frame in store. Refusing to overwrite data. If you want to overwrite data in store, set overwrite=True.
qsprpred - DEBUG - Adding target property 'pchembl_value_Mean' to dataset.
qsprpred - DEBUG - Restoring training data...
qsprpred - DEBUG - Training data restored.
qsprpred - DEBUG - Training features shape: (3265, 0)
qsprpred - DEBUG - Test set features shape: (817, 0)
qsprpred - DEBUG - Training labels shape: (3265, 1)
qsprpred - DEBUG - Test set labels shape: (817, 1)
qsprpred - DEBUG - Training features indices: Index(['LoggingTutorialDataset_0000', 'LoggingTutorialDataset_0001',
       'LoggingTutorialDataset_0002', 'LoggingTutorialDataset_0003',
       'LoggingTutorial

In [3]:
# set the level back to warnings only

logger.setLevel(logging.WARNING)
setLogger(logger)

The `enable_file_logger` utility function will
create a file logger that will write to the file `TutorialLogging.log` in the `tutorial_output` directory.

In [3]:
# logging settings
from qsprpred.logs import logger
import logging
from qsprpred.logs import setLogger
from qsprpred.logs.utils import enable_file_logger

logSettings = enable_file_logger(
    log_folder="../../tutorial_output/",
    filename="TutorialLogging.log",
    debug=False,
    disable_existing_loggers=False
)

A utility function is provided to save your conda environment to a file. This is useful if you want to share your code with others, or if you want to recreate your environment at a later time.

In [13]:
from qsprpred.logs.utils import export_conda_environment

# save the environment to a yaml file
export_conda_environment("../../tutorial_output/TutorialLoggingEnvironment.yml")

Environment exported to ../../tutorial_output/TutorialLoggingEnvironment.yml successfully!


Below we run a standard data preparation and modelling workflow to show logging output.

In [14]:
from qsprpred.data.descriptors.fingerprints import MorganFP
from qsprpred.data import RandomSplit
from qsprpred.data import QSPRDataset

import os

os.makedirs("../../tutorial_output/data", exist_ok=True)

dataset = QSPRDataset.fromTableFile(
    filename="../../tutorial_data/A2A_LIGANDS.tsv",
    store_dir="../../tutorial_output/data",
    name="LoggingTutorialDataset",
    target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
    random_state=42
)

dataset.prepareDataset(
    split=RandomSplit(test_fraction=0.2, dataset=dataset),
    feature_calculators=[MorganFP(radius=3, nBits=2048)],
)

_ = dataset.save()

In [15]:
# Standard model training
from qsprpred.models import SklearnModel
from sklearn.neighbors import KNeighborsRegressor
from qsprpred.models import CrossValAssessor, TestSetAssessor

os.makedirs("../../tutorial_output/models", exist_ok=True)

model = SklearnModel(
    base_dir="../../tutorial_output/models",
    alg=KNeighborsRegressor,
    name="TutorialLoggingModel"
)

CrossValAssessor("r2")(model, dataset)
TestSetAssessor("r2")(model, dataset)

model.fitDataset(dataset)

_ = model.save()

qsprpred - WARNING - Random state supplied, but alg <class 'sklearn.neighbors._regression.KNeighborsRegressor'> does not support it. Ignoring this setting.
Random state supplied, but alg <class 'sklearn.neighbors._regression.KNeighborsRegressor'> does not support it. Ignoring this setting.
